In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import json
from typing import Tuple, List

In [ ]:
%pip install wandb
import wandb

wandb.login()

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False


torch.cuda.set_device(0)
device = torch.device("cuda")


In [ ]:
!gdown 1AuCGKE9nzNjGfvo4A5CX-JdkvWPfivUV # download dataframe.pkl

columns_to_drop = ["song_id", " valence_std", " arousal_std"]
test_fraction = 0.2
random_state = 200

data =pd.read_pickle("/content/dataframe.pkl")
data = data.drop(columns=columns_to_drop)
data[data.columns[[0, 1]]] = data[data.columns[[0, 1]]].div(10)  # scale labels to [0.1-0.9]

test = data.sample(frac=test_fraction, random_state=random_state)
train = data.drop(test.index)

features = train[train.columns[2:]]  # normalize all, except labels
train[train.columns[2:]] = (features - features.mean()) / features.std()

test[test.columns[2:]] = (test[test.columns[2:]] - features.mean()) / features.std()

train_data = train[train.columns[2:]]
train_labels = train[train.columns[:2]]  # valence, arousal
test_data = test[test.columns[2:]]
test_labels = test[test.columns[:2]]
data

In [ ]:
!gdown 1rD8kwaVtWv1jihqeaP9FnqpE61ZTmoXt # download config
sweep_config = json.load(open("/content/dnn_wandb_config.json"))
sweep_config

In [ ]:
def build_dataset(batch_size: int) -> Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_data.values.astype(float)), torch.from_numpy(train_labels.values.astype(float)))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    
    test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_data.values.astype(float)), torch.from_numpy(test_labels.values.astype(float)))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

    return train_loader, test_loader

def build_layer(input_size: int, output_size: int, activation: str, dropout: float, batch_norm: bool) -> List[nn.Module]:
    
    modules = [nn.Linear(input_size, output_size, bias=not batch_norm)]

    if activation == "relu":
        activation = nn.ReLU()
    elif activation == "tanh":
        activation = nn.Tanh()
    elif activation == "leakyrelu":
        activation = nn.LeakyReLU()
    elif activation == "sigmoid":
        activation = nn.Sigmoid()
    modules.append(activation)
    
    if dropout != 0.0:
        modules.append(nn.Dropout(p=dropout))
        
    if batch_norm:
        modules.append(nn.BatchNorm1d(output_size))
        
    return modules

def build_network(input_size: int, output_size: int, mult: float, max_hidden: int, layers_amount: int, activation: str, dropout: float, batch_norm: bool) -> nn.Sequential:
    modules = []
    in_size = input_size
    for i in range(layers_amount-1):
        if mult > 5: # mult > 5 means that we want all layers to have same, given size
            out_size = mult
        else: # otherwise if number of layer i < half the amount of layers, then we want next layer to have this_layer_neurons_amount * mult   else    / mult
            if i < layers_amount/2:
                out_size = in_size*mult
            else:
                out_size = in_size/mult
            out_size = min(out_size, max_hidden)

        layer = build_layer(int(in_size), int(out_size), activation, dropout, batch_norm)

        for module in layer:
            modules.append(module)
        
        in_size = out_size
    
    modules.append(nn.Linear(int(in_size), output_size, bias=not batch_norm)) # add output layer

    network = nn.Sequential(*modules)
    
    return network.to(device)
        

def build_optimizer(network: nn.Sequential, optimizer: str, learning_rate: float) -> optim.Optimizer:
    if optimizer == "adam":
        optimizer = optim.Adam(network.parameters(), lr=learning_rate)
    elif optimizer == "adamw":
        optimizer = optim.AdamW(network.parameters(), lr=learning_rate)
    return optimizer

def build_loss_func(loss_func: str) -> nn.Module:
    if loss_func == "mse":
        loss_func = nn.MSELoss()
    elif loss_func == "huber":
        loss_func = nn.HuberLoss()
    elif loss_func == "l1":
        loss_func = nn.L1Loss()
    return loss_func

def train_epoch(model: nn.Sequential, loader: torch.utils.data.DataLoader, optimizer: optim.Optimizer, loss_func: nn.Module) -> float:
    model.train()
    cumu_loss = 0
    for data, target in loader:
        data, target = data.float().to(device), target.float().to(device)
        optimizer.zero_grad()

        loss = loss_func(model(data), target)
        cumu_loss += loss.item()

        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [ ]:
def calculate_metric(max_diff_arr: List[float], loader: torch.utils.data.DataLoader, model: nn.Sequential) -> List[float]:
    model.eval()
    res = []
    with torch.no_grad():
        for max_diff in max_diff_arr:
            num_preds = 0.
            true_preds = 0.
            for data_inputs, data_labels in loader:
                data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
                preds = model(data_inputs.float())
                preds = preds.cpu().detach().numpy()
                labels = data_labels.cpu().detach().numpy()
                for i in range(len(preds)):
                    if (np.abs(preds[i][0] - labels[i][0]) <= max_diff) and (np.abs(preds[i][1] - labels[i][1]) <= max_diff):
                        true_preds += 1
                    num_preds+=1
            res.append(true_preds/num_preds)
    return res

def train(config=None) -> None:
    # Initialize a new wandb run
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
        config = wandb.config

        train_loader, test_loader = build_dataset(config.batch_size)
        network = build_network(config.input_size, config.output_size, config.mult, config.max_hidden,
                                config.layers_amount, config.activation, config.dropout, config.batch_norm)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        loss_func = build_loss_func(config.loss_func)

        best_acc = 0 #best accuracy for 0.05 max diff
        max_diffs = [0.01, 0.03, 0.05]
        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, train_loader, optimizer, loss_func)
            wandb.log({"loss": avg_loss, "epoch": epoch})

            metric_eval = calculate_metric(max_diffs, test_loader, network)
            if metric_eval[2] > best_acc:
                best_acc = metric_eval[2]
            for max_diff, acc in zip(max_diffs, metric_eval):
                wandb.log({f"acc_{max_diff}_eval": acc})

            metric_train = calculate_metric(max_diffs, train_loader, network)
            for max_diff, acc in zip(max_diffs, metric_train):
                wandb.log({f"acc_{max_diff}_train": acc})
        wandb.log({f"best_accuracy_eval": best_acc})


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, entity = "rozpoznawanie_emocji_olejnik", project="dnn_refactor_5")

In [ ]:
wandb.agent(sweep_id, train, count=25)